# TensorFlow 2.0

In [10]:
from glob import glob
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets 

import matplotlib.pyplot as plt
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

# Data Preprocess

In [7]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

In [8]:
test_df.head()

,name,class
0,../dataset/cifar/test\0_cat.png,cat
1,../dataset/cifar/test\1000_dog.png,dog
2,../dataset/cifar/test\1001_airplane.png,airplane
3,../dataset/cifar/test\1002_ship.png,ship
4,../dataset/cifar/test\1003_deer.png,deer


In [11]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.3,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255,
)

In [14]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col = 'name',
    y_col = 'class',
    target_size = input_shape[:2],
    batch_size = batch_size
)

validation_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col = 'name',
    y_col = 'class',
    target_size = input_shape[:2],
    batch_size = batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [3]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [16]:
model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps = len(validation_generator)
)

C:\Users\jumin\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1563/1563 [==============================] - 322s 206ms/step - loss: 1.7742 - accuracy: 0.3448 - val_loss: 1.4169 - val_accuracy: 0.4866
Epoch 2/10
1563/1563 [==============================] - 253s 162ms/step - loss: 1.4987 - accuracy: 0.4551 - val_loss: 1.2670 - val_accuracy: 0.5374
Epoch 3/10
1563/1563 [==============================] - 273s 174ms/step - loss: 1.3891 - accuracy: 0.4998 - val_loss: 1.2793 - val_accuracy: 0.5375
Epoch 4/10
1563/1563 [==============================] - 225s 144ms/step - loss: 1.3292 - accuracy: 0.5213 - val_loss: 1.1884 - val_accuracy: 0.5653
Epoch 5/10
1563/1563 [==============================] - 228s 146ms/step - loss: 1.2831 - accuracy: 0.5384 - val_loss: 1.1537 - val_accuracy: 0.5835
Epoch 6/10
1563/1563 [==============================] - 231s 148ms/step - loss: 1.2543 - accuracy: 0.5502 - val_loss: 1.2511 - val_accuracy: 0.5580
Epoch 7/10
1563/1563 [==============================] - 233s 149ms/step - loss: 1.2285 - accuracy: 0.5589 - val_